# jieba分词

## 安装

```
conda create -n nlp_py3 python=3.6
source activate nlp_py3
pip install jieba
```

## 特点
* 支持三种分词模式：

    * 精确模式，试图将句子最精确地切开，适合文本分析；
    * 全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义；
    * 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。
* 支持繁体分词
* 支持自定义词典
* MIT 授权协议

## 主要功能
### 1. 分词
* jieba.cut 方法接受三个输入参数: 需要分词的字符串；cut_all 参数用来控制是否采用全模式；HMM 参数用来控制是否使用 HMM 模型
* jieba.cut_for_search 方法接受两个参数：需要分词的字符串；是否使用 HMM 模型。该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细
* jieba.lcut 以及 jieba.lcut_for_search 直接返回 list

In [2]:
# encoding=utf-8
import jieba
seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/gd/ph22ypxj2y31sqdvb08419sw0000gn/T/jieba.cache
Loading model cost 0.727 seconds.
Prefix dict has been built succesfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学


In [7]:
seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Accurate Mode: " + "/ ".join(seg_list))  # 精确模式

Accurate Mode: 我/ 来到/ 北京/ 清华大学


In [6]:
seg_list = jieba.cut("我来到北京清华大学")  # 默认是精确模式
print("Default Model: " + "/ ".join(seg_list))

Default Model: 我/ 来到/ 北京/ 清华大学


In [8]:
seg_list = jieba.cut_for_search("我来到北京清华大学") # 搜索引擎模式
print(", ".join(seg_list))

我, 来到, 北京, 清华, 华大, 大学, 清华大学


### 2.添加自定义词典

* 开发者可以指定自己自定义的词典，以便包含 jieba 词库里没有的词。虽然 jieba 有新词识别能力，但是自行添加新词可以保证更高的正确率
* 用法： jieba.load_userdict(file_name) # file_name 为文件类对象或自定义词典的路径
* 词典格式和 dict.txt 一样，一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。
* 词频省略时使用自动计算的能保证分出该词的词频。

In [36]:
import jieba
jieba.load_userdict("userdict.txt")
test_sent = "李小福是创新办主任也是云计算方面的专家"
words = jieba.cut(test_sent)
print('/'.join(words))
# print(jieba.suggest_freq("创新办", True))

李小福/是/创新办/主任/也/是/云计算/方面/的/专家


* 动态调整词

In [41]:
string = "如果放到post中将出错。"
words = jieba.cut(string)
print('/'.join(words))

如果/放到/post/中将/出错/。


In [42]:
jieba.suggest_freq(('中', '将'), True)
print('/'.join(jieba.cut('如果放到post中将出错。', HMM=False)))

如果/放到/post/中/将/出错/。


In [43]:
print('/'.join(jieba.cut('「台中」正确应该不会被切开', HMM=False)))

「/台/中/」/正确/应该/不会/被/切开


In [44]:
jieba.suggest_freq('台中', True)
print('/'.join(jieba.cut('「台中」正确应该不会被切开', HMM=False)))

「/台中/」/正确/应该/不会/被/切开


* tips: 

P(台中) ＜ P(台)×P(中)，“台中”词频不够导致其成词概率较低

解决方法：强制调高词频

jieba.add_word('台中') 或者 jieba.suggest_freq('台中', True)